# Counters, statemachines and registers

Standard ripple counters are simply implemented using a `intbv` or `modbv` wire type signal that is incremented or decremented within a clocked process. We combine this with a simple state machine that uses enum types.

In [1]:
from cyhdl import *

@block
def fsm_counter_unit(clk   : ClkSignal,
                     en    : Signal.Type(bool),
                     reset : ResetSignal,
                     q     : Signal.Type(bool).Output,
                     *,
                     C_END : int,
                     C_BLANK_END : int
                    ):
    
    t_state = enum("RESET", "IDLE", "RUNNING", "BLANK")
    state = Signal(t_state.RESET)
    
    c = Signal(intbv(0)[12:])
    
    assertion = q.set(state == t_state.RUNNING)
    
    @always_seq(clk.posedge, reset)
    def fsm():
        if state == t_state.RESET:
            state.next = t_state.IDLE
        elif state == t_state.IDLE:
            if en == True:
                state.next = t_state.RUNNING
        elif state == t_state.RUNNING:
            c.next = c + 1
            if c == C_END:
                state.next = t_state.BLANK
        else: # state == t_state.BLANK:
            c.next = c + 1
            if c == C_BLANK_END:
                state.next = t_state.IDLE
                c.next = 0
    
    return instances()
                

The test bench decorator below creates a particular, simulator specific test bench for the encapsulated `@block`. We choose ICARUS verilog this time:

In [2]:
from cyrite.simulation import sim, icarus

@sim.testbench(icarus.ICARUS, 'ns')
@block
def tb_fsm():
    clk = ClkSignal()
    reset = ResetSignal(True, True)
    en, q = [ Signal(bool()) for _ in range(2) ]
    
    @always(delay(1))
    def clkgen():
        clk.next = ~clk
        
    uut = fsm_counter_unit(clk = clk, en = en, reset = reset, q = q,
                          C_END = 24-1, C_BLANK_END = 24 + 8-2)
    
    
    @sequence
    def main():
        reset.next = True
        yield 20 * (clk.posedge, )
        reset.next = False
        yield delay(20)
        en.next = True
        yield delay(2000)
        raise StopSimulation
        
    return instances()


Create an instance of the test bench and run it:

In [3]:
tb = tb_fsm()
tb.run(200, wavetrace = "test.vcd")

 Writing 'fsm_counter_unit' to file /tmp/myirl_tb_fsm_ouxpg8s5/fsm_counter_unit.v 
DEBUG Fallback wire for reset
 Writing 'tb_fsm' to file /tmp/myirl_tb_fsm_ouxpg8s5/tb_fsm.v 
 Creating library file module_defs.v 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/cyrite/simulation/icarus.py:51: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")


0

A wave file trace [[Download]](test.vcd) was created.